In [1]:
import os
import boto3
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import random
import matplotlib.pyplot as plt
import numpy as np
import PIL
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

from tensorflow.keras.applications.vgg19 import (VGG19, preprocess_input, decode_predictions)
from tensorflow.keras.applications import MobileNet
from config import aws_access_key_id, aws_secret_access_key

ModuleNotFoundError: No module named 'config'

In [ ]:
# Set your AWS credentials and region
aws_access_key_id = aws_access_key_id
aws_secret_access_key = aws_secret_access_key 
region_name = "us-east-2"
bucket_name = "capstone-pill-images"
# train_prefix = "dataset/train/"
# validation_prefix = "dataset/valid/"
data_prefix = "data/data"


# Create an S3 client
s3 = boto3.client(
    's3',
     aws_access_key_id=aws_access_key_id,
     aws_secret_access_key=aws_secret_access_key,
     region_name=region_name
    )

In [ ]:

# Function to download images from S3 bucket to a local directory with subdirectories
def download_images_from_s3(bucket_name, prefix, local_directory):
    response = s3.list_objects_v2(Bucket=bucket_name, Prefix=prefix)
    for obj in response.get('Contents', []):
        key = obj['Key']

        # Extract the drug name from the key
        # Split the key at the second "/" to get the correct drug name

    for obj in response.get('Contents', []):
        key = obj['Key']
        local_path = os.path.join(local_directory, os.path.relpath(key, prefix))  # Use os.path.relpath to get the relative path
        os.makedirs(os.path.dirname(local_path), exist_ok=True)  # Create intermediate directories if they don't exist
        s3.download_file(bucket_name, key, local_path)





In [ ]:
# Download training images
local_drug_directory = "local_drug_directory"
download_images_from_s3(bucket_name, data_prefix, local_drug_directory)


In [ ]:
# define loading parameters
img_height = 224
img_width = 224
batch_size = 1

In [ ]:
import splitfolders
splitfolders.ratio('local_drug_directory', output="output", seed=1337, ratio=(.8, 0.1,0.1)) 

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255)
valid_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
train_dir = "output/train/"
valid_dir = "output/val/"

In [ ]:
train_generator = train_datagen.flow_from_directory(train_dir,
                                                    target_size=(img_width,img_height),
                                                    batch_size=batch_size,
                                                    shuffle=True,
                                                    class_mode="categorical"
                                                   )

In [ ]:
validation_generator = valid_datagen.flow_from_directory(valid_dir,
                                                    target_size=(img_width,img_height),
                                                    batch_size=batch_size,
                                                    shuffle=True,
                                                    class_mode="categorical"
                                                   )

In [ ]:
class_names = train_generator.class_indices.keys()
class_names

In [ ]:
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, GlobalAveragePooling2D

In [ ]:
base_model = MobileNet(weights = 'imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False
base_model.summary()

In [ ]:
model = Sequential()

model.add(base_model)
model.add(GlobalAveragePooling2D())
model.add(Dense(23, activation="softmax"))
model.summary()

In [ ]:
model.compile(optimizer='adam',
              loss="categorical_crossentropy",
              metrics=['accuracy'])

In [ ]:
model.fit(train_generator, epochs=2)

In [ ]:
model_loss, model_accuracy = model.evaluate(validation_generator, verbose=2)
print(f" loss : {round(model_loss)*100}%, accuray : {round(model_accuracy,2)*100}%")

In [ ]:
# model.save('MobileNet_02.keras')

In [ ]:

def show_sample_images(local_directory):
    # Go into the local directory and put all subdirectories in a List
    subdirectories = [d for d in os.listdir(local_directory) if os.path.isdir(os.path.join(local_directory, d))]

    # Check if there are any subdirectories
    if not subdirectories:
        print("No subdirectories found in the directory.")
        return

    # Collect all image files from all subdirectories
    all_image_files = []
    for subdirectory in subdirectories:
        subdirectory_path = os.path.join(local_directory, subdirectory)

        # List all image files in the subdirectory
        image_files = [f for f in os.listdir(subdirectory_path) if f.lower().endswith(('.jpg', '.jpeg'))]

        # Add the image files to the combined list
        all_image_files.extend(os.path.join(subdirectory_path, f) for f in image_files)

    # Shuffle the combined list of image files
    random.shuffle(all_image_files)

    # Set the number of random images to display
    num_images_to_display = 5

    # Create subplots to display images side by side
    fig, axes = plt.subplots(1, num_images_to_display, figsize=(15, 3))  # Adjust figsize as needed

    # Loop to select and display 5 random images
    for i in range(num_images_to_display):
        # Check if there are still image files available
        if not all_image_files:
            break

        # Pop a random image from the combined list
        random_image_file = all_image_files.pop()

        # Load and display the randomly selected image
        img = mpimg.imread(random_image_file)

        # Display the image in the subplot
        axes[i].imshow(img)
        axes[i].axis('off')  # Turn off axis labels

    # Show the subplots
    plt.show()

# Example usage
show_sample_images(local_drug_directory)